In [0]:
!pip install pymorphy2

In [180]:
from pymorphy2 import MorphAnalyzer
from lxml import html
import nltk
nltk.download('stopwords')
from nltk import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
import gensim
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from collections import Counter, defaultdict
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))


def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return ' '.join(words)


def tokenize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return ' '.join(words)

  

Часть 1

Буду обучать w2v модель на основе собственноручно собранного корпуса статей РБК за 2019 г. (там около 51000 статей)

In [225]:
from google.colab import files
uploaded = files.upload()

Saving RBC_fin.txt to RBC_fin.txt


In [0]:
import zipfile
zipcorpus = zipfile.ZipFile('RBC_fin.zip')
zipcorpus.extractall()
zipcorpus.close

<bound method ZipFile.close of <zipfile.ZipFile filename='RBC_fin.zip' mode='r'>>

In [0]:
data = open('RBC_fin.txt').read().splitlines()

data_norm = [normalize(text) for text in data]

In [0]:
data_norm = [text for text in data_norm if text]

In [230]:
data_norm[:5]

['авария воронежский область погибнуть управлять директор оэмк дтп воронежский область погибнуть управлять директор оскольский элекрометаллургический комбинат оэмк александр тищенко водитель предприятие василий юмашев',
 'говориться сообщение холдинг металлоинвест рбк входить оэмк',
 'директор металлоинвест андрей варичев назвать смерть тищенко огромный утрата компания также выразить соболезнование семья погибший',
 'отмечаться тищенко 1980 год работать новолипецкий металлургический завод 2003-го стать заместитель главный инженер оэмк технический перевооружение',
 '2011 год главное инженер комбинат декабрь 2018 год стать управлять директор']

In [0]:
len(data_norm)

845659

In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)

In [0]:
w2v.save("word2vec.model")

In [9]:
#from google.colab import files
#uploaded = files.upload()

Saving word2vec.model to word2vec.model


In [0]:
#w2v = gensim.models.Word2Vec.load('word2vec.model')

Скачаем предуобученную модель с Rusvectores: tayga_upos_skipgram_300_2_2019

In [7]:
from google.colab import files
uploaded = files.upload()

Saving 185.zip to 185.zip


In [0]:
import zipfile
zipcorpus = zipfile.ZipFile('185.zip')
zipcorpus.extractall()
zipcorpus.close

In [0]:
taiga = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

Скачаем данные соревнования по определению перефразирования с колонками нормализованных в mystem текстов с частереченой разметкой

In [26]:
from google.colab import files
uploaded = files.upload()

Saving data_paraphraser_norm.csv to data_paraphraser_norm.csv


In [0]:
data = pd.read_csv('data_paraphraser_norm.csv')

Нормализуем тексты также без частеречной разметки, т.к. в обученную на РБК модель подавались слова в начальной форме без тэгов

In [0]:
data['text_1_norm_no_pos'] = data['text_1'].apply(normalize)
data['text_2_norm_no_pos'] = data['text_2'].apply(normalize)

In [234]:
data.head(5)

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_norm_no_pos,text_2_norm_no_pos
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


Функция векторизации предложения/текста

In [0]:
def get_embedding(model, text, dim):
  text = text.strip(punct).split()
  words = Counter(text)
  total = len(text)
  vectors = np.zeros((len(words), dim))
  for i,word in enumerate(words):
    try:
      v = model[word]
      vectors[i] = v * (words[word]/total)
    except (KeyError, ValueError):
      #print('Что-то пошло не так')
      continue
    
  if vectors.any():
    vector = np.average(vectors, axis=0)
  else:
    vector = np.zeros((dim))
  return vector

In [0]:
get_embedding(w2v, 'слово слово текст работать функция', w2v.vector_size)

In [0]:
get_embedding(taiga, 'слово_NOUN слово_NOUN текст_NOUN работать_VERB функция_NOUN', taiga.vector_size)

Функция создания списка векторизованных текстов, на основе входящего списка нормализованных текстов и модели

In [0]:
def vectorized_column (column, model):
  dim = model.vector_size
  vectorized = []
  for text in column:
    vectorized += [get_embedding(model, text, dim)]
  return vectorized

Предскажем перефразирование на основе модели РБК

In [0]:
X_text_1 = vectorized_column (data['text_1_norm_no_pos'], w2v)
X_text_2 = vectorized_column (data['text_2_norm_no_pos'], w2v)
X_text = np.concatenate([X_text_1, X_text_2], axis = 1)

In [0]:
y = data.label.values

In [0]:
clf = LogisticRegression(max_iter=200) 

In [0]:
cv_scores_RBC = cross_val_score(clf, X_text, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_scores_RBC)
print('average cv_score:', np.mean(cv_scores_RBC))

cv scores: [0.45228216 0.48616874 0.5266436  0.40484429 0.40276817]
average cv_score: 0.4545413908790268


А теперь посмотрим как предсказывается перефразирование на основе модели корпуса Тайги

In [0]:
X_text_1 = vectorized_column (data['text_1_norm'], taiga)
X_text_2 = vectorized_column (data['text_2_norm'], taiga)
X_text = np.concatenate([X_text_1, X_text_2], axis = 1)

In [0]:
cv_scores_Taiga = cross_val_score(clf, X_text, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_scores_Taiga)
print('average cv_score:', np.mean(cv_scores_Taiga))

cv scores: [0.46749654 0.50069156 0.50242215 0.40692042 0.39307958]
average cv_score: 0.4541220500892571


Результаты обеих моделей очень близки. Модель РБК оказалась даже немного лучше. Учитывая, что размер корпуса статей РБК за 2019 г.(51000 статей, если каждая из них по 100-200 слов, то это ~5-10 млн. слов) гораздо меньше корпуса Тайги(5 млрд. слов), результат скорее говорит в пользу первой модели. Парафразы в соревновании были взяты из новостных текстов, соответственно корпус новостных текстов лучше подходит для этой конкретной задачи. Могу предположить, что если была бы обучена  модель на основе корпуса художественной литературы (сопоставимого с РБК корпусом по размерам), то результат был бы хуже. Зная заранее тематику новостей в парафразах, можно было бы обучить модель на основе соответствующей рубрики (например, "Экономика" или "Латинская Америка") и улучшить результат.

Часть 2

In [0]:
clf = LogisticRegression(max_iter=200)

In [450]:
tfidf = TfidfVectorizer(min_df=0, max_df=0.25, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.25, max_features=1000,
                min_df=0, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
y = data.label.values

Уменьшение min_df дает лучший результат, также менял max_df до того момента, пока f1 не начал уменьшаться

SVD

Пробовал менять число компонентов и итераций

In [0]:
svd = TruncatedSVD(999, n_iter=20, random_state=42)
X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm']))

In [468]:
svd_sim = []
for a, b in zip(X_text_1_svd, X_text_2_svd):
      svd_sim += [cosine_similarity([a], [b])[0]]
data['svd_sim'] = svd_sim
svd_sim[:5]

[array([-0.00479332]),
 array([0.05444651]),
 array([-0.04192538]),
 array([0.0641008]),
 array([0.05627796])]

In [469]:
cv_sim = cross_val_score(clf, svd_sim, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_sim)
print('average cv_score:', np.mean(cv_sim))
#Здесь некотоыре полученные средние значения: 0.409856279343565, 0.4073635898098561, 0.4111008054674152

cv scores: [0.39142462 0.40871369 0.41453287 0.42975779 0.43114187]
average cv_score: 0.4151141677076005


NMF

Пробовал менять параметры, результаты скорее ухудшались либо не менялись

In [0]:
nmf = NMF(50, init = 'nndsvda', alpha = 0.1,  random_state=42)
X_text_1_nmf = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.fit_transform(tfidf.transform(data['text_2_norm']))

In [528]:
nmf_sim = []
for a, b in zip(X_text_1_nmf, X_text_2_nmf):
      nmf_sim += [cosine_similarity([a], [b])[0]]
data['nmf_sim'] = nmf_sim
nmf_sim[:5]

[array([0.36894177]),
 array([0.19017315]),
 array([0.03695237]),
 array([0.00908225]),
 array([0.00145757])]

In [529]:
nmf_sim = cross_val_score(clf, nmf_sim, y, scoring='f1_micro', cv=5)
print('cv scores:', nmf_sim)
print('average cv_score:', np.mean(nmf_sim))
#Здесь некоторые полученные средние значения: 0.4091600262267464, 0.3994714449118676,0.39960985321636583

cv scores: [0.40940526 0.40940526 0.40899654 0.40899654 0.40899654]
average cv_score: 0.4091600262267464


W2v. Taiga

In [0]:
X_text_1_taiga = vectorized_column (data['text_1_norm'], taiga)
X_text_2_taiga = vectorized_column (data['text_2_norm'], taiga)
X_text_taiga = np.concatenate([X_text_1_taiga, X_text_2_taiga], axis = 1)

In [0]:
taiga_sim = []
for a, b in zip(X_text_1_taiga, X_text_2_taiga):
      taiga_sim += [cosine_similarity([a], [b])[0]]
taiga_sim[:5]
data['taiga_sim'] = taiga_sim

In [258]:
cv_scores_taiga = cross_val_score(clf, taiga_sim, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_scores_taiga)
print('average cv_score:', np.mean(cv_scores_taiga))

cv scores: [0.57883817 0.60235131 0.62076125 0.50519031 0.50380623]
average cv_score: 0.5621894547421116




W2V. RBC


In [259]:
X_text_1_RBC = vectorized_column (data['text_1_norm_no_pos'], w2v)
X_text_2_RBC = vectorized_column (data['text_2_norm_no_pos'], w2v)
X_text_RBC = np.concatenate([X_text_1_RBC, X_text_2_RBC], axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
RBC_sim = []
for a, b in zip(X_text_1_RBC, X_text_2_RBC):
      RBC_sim += [cosine_similarity([a], [b])[0]]
RBC_sim[:5]
data['RBC_sim'] = RBC_sim

In [261]:
cv_scores_RBC = cross_val_score(clf, RBC_sim, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_scores_RBC)
print('average cv_score:', np.mean(cv_scores_RBC))

cv scores: [0.58298755 0.59820194 0.63460208 0.47820069 0.51141869]
average cv_score: 0.5610821883061254


Fastext

In [0]:
#fast_text = gensim.models.FastText([text.split() for text in data_norm], size=50) 

In [262]:
X_text_1_ft = vectorized_column (data['text_1_norm_no_pos'], fast_text)
X_text_2_ft = vectorized_column (data['text_2_norm_no_pos'], fast_text)
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [267]:
ft_sim = []
for a, b in zip(X_text_1_ft, X_text_2_ft):
      ft_sim += [cosine_similarity([a], [b])[0]]
data['ft_sim'] = ft_sim
ft_sim[:5]

[array([0.90769982]),
 array([0.88987209]),
 array([0.92615769]),
 array([0.5602007]),
 array([0.71105018])]

In [264]:
cv_scores_ft = cross_val_score(clf, ft_sim, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_scores_ft)
print('average cv_score:', np.mean(cv_scores_ft))

cv scores: [0.55463347 0.57538036 0.59031142 0.47612457 0.46228374]
average cv_score: 0.5317467108884071


In [530]:
data.head(1)

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_norm_no_pos,text_2_norm_no_pos,svd_sim,RBC_sim,nmf_sim,taiga_sim,ft_sim
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,[-0.004793321888195275],[0.9388591548521453],[0.3689417652572511],[0.870368706403491],[0.9076998179237329]


Посмотрим, как будет работать модель, если учитывать все 5 колонок косинусной близости

In [531]:
X = data.iloc[:, 7:12].values
final_score = cross_val_score(clf, X, y, scoring='f1_micro', cv=5)
print('cv scores:', final_score)
print('average cv_score:', np.mean(final_score))

cv scores: [0.58506224 0.62378976 0.64913495 0.50588235 0.53771626]
average cv_score: 0.5803171139092689


Лучший результат показало предсказание на основе комбинированного использования 5 разных методов. Если рассматривать методы отдельно, то лучше всего с задачей справилось предсказание на основе корпуса "Тайга". w2v на основе корпуса РБК показал результат очень близкий к Тайге, но всё же немного хуже. Fasttext также незначительно хуже вышеперечисленных моделей. SVD и NMF, которые не обучались на сторонних корпусах, оказались значительно хуже (f1 ~ 0.41).